# Aplicaciones de Transformers en Procesamiento de Lenguaje Natural

Este cuaderno explora distintas aplicaciones de los modelos Transformers en tareas de procesamiento de lenguaje natural (PLN), utilizando ejemplos prácticos y explicaciones teóricas.

---

## ¿Qué es la arquitectura Transformer?

La **arquitectura Transformer** es un tipo de red neuronal propuesta por Vaswani et al. en 2017. Su principal innovación es el mecanismo de *atención*, que permite al modelo enfocarse en diferentes partes de la entrada para procesar información contextual de manera eficiente. Los Transformers reemplazaron a modelos anteriores como LSTM y GRU en muchas tareas de PLN debido a su capacidad para manejar secuencias largas y aprender dependencias complejas.

---

## ¿Qué es la librería Transformers de Hugging Face?

La **librería Transformers de Hugging Face** es una biblioteca de Python que proporciona implementaciones preentrenadas de modelos basados en la arquitectura Transformer (como BERT, GPT, T5, etc.) para tareas de PLN. Permite cargar modelos fácilmente y utilizarlos en tareas como clasificación, traducción, resumen, etc., sin necesidad de entrenarlos desde cero. Podes explorar los modelos disponibles en la [página de modelos de Hugging Face](https://huggingface.co/models).

---

## ¿Qué es un pipeline en Hugging Face?

Un **pipeline** en Hugging Face es una interfaz de alto nivel que simplifica el uso de modelos preentrenados para tareas específicas. Permite ejecutar tareas como clasificación de texto, reconocimiento de entidades, traducción, etc., con solo unas pocas líneas de código, ocultando detalles complejos de preprocesamiento y postprocesamiento.


## 1. Definir el texto de ejemplo

A continuación, se presenta un texto de ejemplo que será utilizado en las distintas aplicaciones de modelos Transformers. Este texto simula una queja de un cliente sobre un pedido equivocado, lo que nos permitirá explorar tareas como clasificación, reconocimiento de entidades, respuesta a preguntas, resumen, traducción y generación de texto.

In [2]:
texto = """Querido MercadoLibre, la semana pasada pedí una figura de acción de Optimus Prime desde su tienda online.
Para mi sorpresa, cuando abrí el paquete, descubrí horrorizado que me habían enviado una figura de Megatron.
Como fan de los Autobots, espero que entiendan mi decepción. Solicito un cambio urgente del producto."""

## 2. Clasificación de texto con Transformers

La **clasificación de texto** es una tarea fundamental en PLN que consiste en asignar una o varias etiquetas a un texto, como por ejemplo identificar el sentimiento (positivo, negativo, neutro) o la intención del mensaje. Los modelos Transformers, gracias a su capacidad para comprender el contexto, demostraron un rendimiento sobresaliente en esta tarea.

Utilizaremos el pipeline `text-classification` de la librería 🤗 Transformers para analizar el texto de ejemplo y visualizar el resultado de la clasificación.

In [3]:
from transformers import pipeline
# Modelo de clasificación de texto (sentimiento) en español
# Puedes buscar otros modelos en https://huggingface.co/models
classifier = pipeline("text-classification", model="pysentimiento/robertuito-sentiment-analysis")
import pandas as pd
outputs = classifier(texto)
pd.DataFrame(outputs)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/925 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/435M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

Device set to use cpu


,label,score
0,NEG,0.913063


## 3. Reconocimiento de entidades nombradas (NER)

El **Reconocimiento de Entidades Nombradas** (NER, por sus siglas en inglés) es la tarea de identificar y clasificar automáticamente nombres propios, lugares, organizaciones y otras entidades relevantes dentro de un texto. Los Transformers permiten realizar NER de manera eficiente y precisa.

A continuación, aplicamos el pipeline `ner` al texto de ejemplo para extraer las entidades nombradas presentes.

In [4]:
# Modelo de reconocimiento de entidades nombradas (NER) en español
# Puedes buscar otros modelos en https://huggingface.co/models
ner_tagger = pipeline("ner", model="mrm8488/bert-spanish-cased-finetuned-ner", aggregation_strategy="simple")
outputs = ner_tagger(texto)
pd.DataFrame(outputs)

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,entity_group,score,word,start,end
0,ORG,0.756585,MercadoLibre,8,20
1,MISC,0.829108,Optimus Prime,68,81
2,MISC,0.763651,Megatron,205,213
3,MISC,0.954693,Autobot,231,238


## 4. Respuesta a preguntas basada en contexto

La tarea de **respuesta a preguntas** (Question Answering) consiste en responder preguntas específicas utilizando un contexto dado. Los modelos Transformers pueden comprender el texto y extraer la información relevante para responder de manera precisa.

En el siguiente ejemplo, preguntamos al modelo qué desea el cliente, utilizando el pipeline `question-answering`.

In [5]:
# Modelo de respuesta a preguntas en español
# Puedes buscar otros modelos en https://huggingface.co/models
reader = pipeline("question-answering", model="PlanTL-GOB-ES/roberta-large-bne-sqac")
question = "¿Qué quiere el cliente?"
outputs = reader(question=question, context=texto)
pd.DataFrame([outputs])

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu


,score,start,end,answer
0,0.999948,285,315,un cambio urgente del producto


## 5. Resumen automático de texto

El **resumen automático** permite condensar la información principal de un texto extenso en una versión más corta, manteniendo el significado esencial. Los Transformers han revolucionado esta tarea gracias a su capacidad de comprensión contextual.

A continuación, utilizamos el pipeline `summarization` para obtener un resumen del texto de ejemplo.

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# Puedes buscar otros modelos en https://huggingface.co/models
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
resumidor = pipeline("summarization", model=model, tokenizer=tokenizer)
resumen = resumidor(texto, max_length=80, min_length=20, do_sample=False)
print(resumen[0]['summary_text'])

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

Device set to use cpu


Hace unos meses, un paquete de acciones de la tienda online Amazon me envió una figura de Megatron.


## 6. Traducción automática del español a inglés

La **traducción automática** es una de las aplicaciones más destacadas de los Transformers, permitiendo traducir textos entre diferentes idiomas con alta calidad. Utilizaremos el pipeline `translation_es_to_en` para traducir el texto de ejemplo del español al inglés.

In [7]:
# Modelo de traducción de español a inglés
# Puedes buscar otros modelos en https://huggingface.co/models
translator = pipeline("translation_es_to_en", model="Helsinki-NLP/opus-mt-es-en")
outputs = translator(texto)
print(outputs[0]['translation_text'])

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


Dear MarketFree, last week I asked for an action figure of Optimus Prime from his online store. To my surprise, when I opened the package, I discovered horrified that I had been sent a figure of Megatron. As a fan of Autobots, I hope you understand my disappointment. I request an urgent change of product.


## 7. Generación automática de texto (respuesta de servicio al cliente)

La **generación automática de texto** permite crear o continuar textos de manera coherente a partir de un prompt inicial. Esta capacidad es útil, por ejemplo, para redactar respuestas automáticas en atención al cliente.

En el siguiente ejemplo, generamos una posible respuesta de servicio al cliente utilizando el pipeline `text-generation`.

In [8]:
# Modelo de generación de texto en español
# Puedes buscar otros modelos en https://huggingface.co/models
generator = pipeline("text-generation", model="datificate/gpt2-small-spanish")
respuesta_inicial = "Estimado cliente, lamentamos mucho lo ocurrido con su pedido. "
prompt = texto + "\n\nRespuesta del servicio al cliente:\n" + respuesta_inicial
outputs = generator(
    prompt,
    max_new_tokens=150,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    repetition_penalty=1.3,
    eos_token_id=50256
)
print(outputs[0]['generated_text'])

config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

Device set to use cpu


Querido MercadoLibre, la semana pasada pedí una figura de acción de Optimus Prime desde su tienda online.
Para mi sorpresa, cuando abrí el paquete, descubrí horrorizado que me habían enviado una figura de Megatron.
Como fan de los Autobots, espero que entiendan mi decepción. Solicito un cambio urgente del producto.

Respuesta del servicio al cliente:
Estimado cliente, lamentamos mucho lo ocurrido con su pedido.  Una vez más, no me importaba más la oferta final y no se leían a él ninguna otra opción para comprar el kit.  En ese momento, ya no puedo decir qué punto debe hacer.
El 30 de abril, durante las celebraciones del aniversario número 100 de "The New 52", fue anunciado que sería la nueva encarnación de Cybertron como parte de una serie animada por Warner Bros., "Los Simpson". Los personajes principales son Iron Man, Samurai Shotgune y T-Moon, así como Boomerang, Ra's al Ghul, Aliens, Scrooge y Ratker. La primera temporada se estrenó originalmente entre el 11 de septiembre y el 20 d

---

## Ejercicios autónomos (40 minutos)

A continuación, te proponemos ejercicios prácticos para que pongas en práctica lo aprendido sobre la arquitectura Transformer, la librería Transformers de Hugging Face y el uso de pipelines en tareas de PLN.

**Recorda:** Para cada ejercicio, explora la [página de modelos de Hugging Face](https://huggingface.co/models) y busca modelos que sean apropiados para español, o incluso para el contexto argentino si los encontras.

### 1. Clasificación de textos propios
- Escribi dos textos breves (pueden ser opiniones, quejas o comentarios sobre servicios o productos en Argentina, por ejemplo sobre colectivos, supermercados, bancos, etc.).
- Busca en Hugging Face un modelo de clasificación de texto en español y usalo con el pipeline `text-classification` para analizar el sentimiento de cada texto.
- ¿El resultado coincide con tu expectativa? Explica brevemente.

### 2. Reconocimiento de entidades en noticias argentinas
- Busca un párrafo de una noticia reciente de un medio argentino (por ejemplo, Clarín, La Nación, Página/12).
- Busca en Hugging Face un modelo de NER para español y aplícalo para identificar entidades nombradas.
- Enumera las entidades encontradas y clasifícalas (persona, organización, lugar, etc.).

### 3. Respuesta a preguntas personalizada
- Escribi un pequeño texto narrativo (4-5 líneas) sobre una situación cotidiana en Buenos Aires (por ejemplo, un reclamo en una oficina pública, una experiencia en el subte, etc.).
- Formula dos preguntas sobre el texto y utiliza un modelo de `question-answering` de Hugging Face para responderlas.
- ¿Las respuestas son correctas y precisas? ¿Por qué?

### 4. Resumen de un texto propio
- Elegí un texto más largo (puede ser un fragmento de Wikipedia sobre Argentina, una noticia, etc.).
- Busca un modelo de resumen en español en Hugging Face y úsalo con el pipeline `summarization`.
- Compara el resumen generado con el texto original: ¿qué información se perdió y cuál se mantuvo?

### 5. Traducción y análisis
- Escribi un texto breve en español rioplatense y tradúcilo al inglés usando un modelo de traducción de Hugging Face.
- Luego, traduci el resultado nuevamente al español (podes usar otro pipeline o Google Translate).
- ¿El texto final es igual al original? ¿Qué diferencias encontras?

### 6. Generación creativa de texto
- Escribe el inicio de una historia o una pregunta abierta relacionada con Buenos Aires.
- Busca un modelo de generación de texto en español en Hugging Face y úsalo con el pipeline `text-generation` para continuar el texto.
- Analiza la coherencia y creatividad de la respuesta generada.

---

**Tip:** Para cada ejercicio, explora la [página de modelos de Hugging Face](https://huggingface.co/models) y lee la descripción de los modelos antes de usarlos. Si tienes dudas, consulta la documentación oficial de [Transformers](https://huggingface.co/docs/transformers/index).

1) Clasificación de Textos

In [9]:
print("\n--- EJERCICIO 1: Clasificación de Textos Propios ---")

# Tus dos textos breves
texto_1 = "El servicio de internet que tengo es cualquier cosa, se corta cada dos por tres y la atención al cliente es malisima, no lo contrato nunca mas."
texto_2 = "La nueva línea de colectivos en CABA es una genialidad, llega justo cuando la app dice."

# Reutilizamos el classifier de sentimiento cargado al inicio (pysentimiento/robertuito-sentiment-analysis)
# Si quieres probar otro modelo, cárgalo aquí:
# from transformers import pipeline
# classifier_nuevo = pipeline("text-classification", model="otro/modelo-de-sentimiento-espanol")

print("\nAnálisis de Sentimiento para Texto 1:")
outputs_1 = classifier(texto_1)
display(pd.DataFrame(outputs_1))

print("\nAnálisis de Sentimiento para Texto 2:")
outputs_2 = classifier(texto_2)
display(pd.DataFrame(outputs_2))



--- EJERCICIO 1: Clasificación de Textos Propios ---

Análisis de Sentimiento para Texto 1:


,label,score
0,NEG,0.977796



Análisis de Sentimiento para Texto 2:


,label,score
0,POS,0.95153


* Reflexión (Ejercicio 1)
Para el Texto 1 (queja sobre internet), esperaba un sentimiento NEGATIVO. El modelo lo clasificó como NEG. Esto coincide.

* Para el Texto Propio 2 (comentario positivo sobre colectivos), esperaba un sentimiento POSITIVO. El modelo lo clasificó como POS. Esto también coincide.

En ambos casos, el modelo fue capaz de identificar correctamente la polaridad emocional de los textos, demostrando su efectividad para analizar opiniones.

2) Reconocimiento de Entidades en Noticias Argentinas

In [11]:
# Párrafo de una noticia argentina (ejemplo ficticio basado en eventos comunes)
noticia = "El presidente Javier Milei se reunió hoy con el gobernador de Buenos Aires, Axel Kicillof, en Casa Rosada para discutir las reformas económicas. La Confederación General del Trabajo (CGT) emitió un comunicado expresando su preocupación. La reunión duró dos horas."

# Reutilizamos el ner_tagger cargado al inicio (mrm8488/bert-spanish-cased-finetuned-ner)
# Si quieres probar otro modelo, cárgalo aquí:
# from transformers import pipeline
# ner_tagger_nuevo = pipeline("ner", model="otro/modelo-ner-espanol", aggregation_strategy="simple")

print("Texto de la noticia:")
print(noticia)

print("\nEntidades nombradas identificadas:")
outputs_ner_noticia = ner_tagger(noticia)
df_ner_noticia = pd.DataFrame(outputs_ner_noticia)
display(df_ner_noticia)

print("\nClasificación de Entidades Encontradas")
# Ejemplo de clasificación manual basada en los resultados del DataFrame:
for _, row in df_ner_noticia.iterrows():
    entity_name = row['word']
    entity_type = row['entity_group']
    print(f"- **{entity_name}**: {entity_type}")

Texto de la noticia:
El presidente Javier Milei se reunió hoy con el gobernador de Buenos Aires, Axel Kicillof, en Casa Rosada para discutir las reformas económicas. La Confederación General del Trabajo (CGT) emitió un comunicado expresando su preocupación. La reunión duró dos horas.

Entidades nombradas identificadas:


,entity_group,score,word,start,end
0,PER,0.998295,Javier Milei,14,26
1,LOC,0.922009,Buenos Aires,62,74
2,PER,0.984237,Axel Kicillof,76,89
3,LOC,0.999344,Casa Rosada,94,105
4,ORG,0.999084,Confederación General del Trabajo,148,181
5,ORG,0.954370,CGT,183,186



Clasificación de Entidades Encontradas
- **Javier Milei**: PER
- **Buenos Aires**: LOC
- **Axel Kicillof**: PER
- **Casa Rosada**: LOC
- **Confederación General del Trabajo**: ORG
- **CGT**: ORG


3) Respuesta a Preguntas Personalizada

In [12]:
contexto_caba = """Hoy fui a la oficina del ANSES en la Avenida Córdoba para tramitar un certificado.
Había una fila larguísima y esperé casi dos horas. Al final, la empleada, una señora llamada Marta, me atendió muy amablemente.
Me dijo que mi trámite estaba incompleto porque faltaba una fotocopia del DNI.
Tendré que volver la semana que viene con todo listo."""

# Reutilizamos el reader cargado al inicio (PlanTL-GOB-ES/roberta-large-bne-sqac)
# Si quieres probar otro modelo, cárgalo aquí:
# from transformers import pipeline
# reader_nuevo = pipeline("question-answering", model="otro/modelo-qa-espanol")

print("Contexto:")
print(contexto_caba)

# Primera pregunta
pregunta_1 = "¿Dónde está la oficina de ANSES?"
print(f"\nPregunta 1: '{pregunta_1}'")
respuesta_1 = reader(question=pregunta_1, context=contexto_caba)
display(pd.DataFrame([respuesta_1]))

# Segunda pregunta
pregunta_2 = "¿Quién atendió al narrador?"
print(f"\nPregunta 2: '{pregunta_2}'")
respuesta_2 = reader(question=pregunta_2, context=contexto_caba)
display(pd.DataFrame([respuesta_2]))

Contexto:
Hoy fui a la oficina del ANSES en la Avenida Córdoba para tramitar un certificado.
Había una fila larguísima y esperé casi dos horas. Al final, la empleada, una señora llamada Marta, me atendió muy amablemente.
Me dijo que mi trámite estaba incompleto porque faltaba una fotocopia del DNI.
Tendré que volver la semana que viene con todo listo.

Pregunta 1: '¿Dónde está la oficina de ANSES?'


,score,start,end,answer
0,0.642998,34,52,la Avenida Córdoba



Pregunta 2: '¿Quién atendió al narrador?'


,score,start,end,answer
0,0.500913,157,181,una señora llamada Marta


Análisis de Respuestas

* Para la pregunta 1 ¿Dónde está la oficina de ANSES?, la respuesta: "Avenida Córdoba" es **correcta y precisa**.

El modelo extrajo directamente la ubicación mencionada en el texto.

* Para la pregunta 2 ¿Quién atendió al narrador?, la respuesta "Marta" es **correcta y precisa**.

El modelo identificó a la persona que realizó la acción de atender.

Esto demuestra la capacidad del modelo QA para localizar y extraer información específica relevante del texto basándose en la pregunta, incluso en un contexto narrativo corto y cotidiano

5) Traducción y Análisis

In [14]:
# Texto en español rioplatense
texto_rioplatense = "Che, qué quilombo la avenida 9 de Julio a esta hora. ¡Un bondi lo complico todo!"

# Reutilizamos el translator es-en cargado al inicio (Helsinki-NLP/opus-mt-es-en)
# Si quieres probar otro modelo, cárgalo aquí:
# from transformers import pipeline
# translator_es_en_nuevo = pipeline("translation_es_to_en", model="otro/modelo-es-en")

print("Texto Original (Rioplatense):")
print(texto_rioplatense)

# 1. Traducir a inglés
traduccion_a_ingles = translator(texto_rioplatense)
texto_en_ingles = traduccion_a_ingles[0]['translation_text']
print("\nTraducción a Inglés:")
print(texto_en_ingles)

# 2. Traducir de inglés a español (usando otro modelo, si disponible, o el mismo en reversa)
# Para este ejemplo, voy a cargar un modelo en-es.
# Si solo tienes uno, el mismo 'Helsinki-NLP/opus-mt-es-en' puede a veces usarse para en-es
# si se le pasa el source_language y target_language, pero es mejor usar uno específico.
# from transformers import pipeline
translator_en_es = pipeline("translation_en_to_es", model="Helsinki-NLP/opus-mt-en-es")

traduccion_de_vuelta_a_espanol = translator_en_es(texto_en_ingles)
texto_final_espanol = traduccion_de_vuelta_a_espanol[0]['translation_text']
print("\nTraducción de vuelta a Español:")
print(texto_final_espanol)

print("\nComparación y Análisis ")
print(f"Texto Original: '{texto_rioplatense}'")
print(f"Texto Final:    '{texto_final_espanol}'")

Texto Original (Rioplatense):
Che, qué quilombo la avenida 9 de Julio a esta hora. ¡Un bondi lo complico todo!

Traducción a Inglés:
Hey, what a quilombo on 9th of July at this hour. A bondi made it all up!


Device set to use cpu



Traducción de vuelta a Español:
Oye, qué quilombo el 9 de julio a esta hora. ¡Un Bondi lo inventó todo!

Comparación y Análisis 
Texto Original: 'Che, qué quilombo la avenida 9 de Julio a esta hora. ¡Un bondi lo complico todo!'
Texto Final:    'Oye, qué quilombo el 9 de julio a esta hora. ¡Un Bondi lo inventó todo!'


¿El texto final es igual al original? Probablemente no

Diferencias encontradas:
- **Pérdida de jerga/expresiones locales**: 'Che' y 'quilombo' son difíciles de traducir directamente y el modelo los reemplaza con equivalentes más formales o genéricos ('Hey', 'mess'/'traffic'). 'Bondi' es traducido como 'bus' que es correcto pero pierde el toque rioplatense.
- **Estructura de la frase**: A veces, el orden de las palabras o la construcción de la frase pueden cambiar ligeramente en la doble traducción.
- **Matices de significado**: Aunque el significado general se mantiene, los matices culturales o el tono informal del original pueden perderse.

Esta 'ida y vuelta' demuestra las limitaciones de la traducción automática, especialmente con el argot o las expresiones idiomáticas. Los modelos son buenos en el significado literal, pero capturar el estilo y los matices culturales es un desafío mayor.

6) Generación Creativa de Texto

In [16]:
# Inicio de una historia o pregunta abierta relacionada con Buenos Aires
historia_caba = "En una tarde lluviosa de Buenos Aires, el pequeño Timy juega tranquilamente a los videos juegos, hasta que de pronto..."

# Reutilizamos el generator cargado al inicio (datificate/gpt2-small-spanish)
# Si quieres probar otro modelo, cárgalo aquí:
# from transformers import pipeline
# generator_nuevo = pipeline("text-generation", model="otro/modelo-generacion-espanol")
# Generar la continuación de la historia
# Aumentamos un poco max_new_tokens para una continuación más larga
outputs_generacion = generator(
    historia_caba,
    max_new_tokens=100, # Más tokens para una historia más larga
    do_sample=True,
    temperature=0.8, # Un poco más de creatividad
    top_k=50,
    top_p=0.9,
    repetition_penalty=1.2, # Penalización de repetición
    eos_token_id=50256
)
print("\nContinuación generada:")
print(outputs_generacion[0]['generated_text'])



Continuación generada:
En una tarde lluviosa de Buenos Aires, el pequeño Timy juega tranquilamente a los videos juegos, hasta que de pronto...

Cuando llega a la ciudad del mismo nombre, Timy se pone en contacto con el famoso músico argentino Héctor Parodi quien le cuenta su historia y lo invita a visitarlo.
Luego de un tiempo de viaje (entre fines de los años 60 e inicios de los 70) Timy se traslada junto al guitarrista Héctor Parodi para tocar varios temas, como «Adolescencia», «La canción de mi madre», o «El amor». El día domingo 31 de mayo, cuando ya



Análisis de la Generación

- El texto generado mantiene la temática del personaje Timy, sin embargo la continuacion de la historia se va de las ramas y no logra una buena conexión.